In [2]:
%load_ext autoreload
%autoreload 2
import os
PATH = os.path.dirname(os.getcwd())
import pandas as pd
import tqdm
import sys
sys.path.append(PATH)
from skills_matcher.scripts.spacy_model import split_data, make_spacy_model
from skills_matcher.scripts.spacy_model import load_results_auto, load_results_manual

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def load_paths(data_name, dictionary_name, train_spacy_name, val_spacy_name):
    PATH_TRAIN_DATA  = f"{PATH}/skills_matcher/models/{train_spacy_name}.spacy"
    PATH_VAL_DATA  = f"{PATH}/skills_matcher/models/{val_spacy_name}.spacy"
    PATH_DICT = f"{PATH}/skills_matcher/data/dictionaries/{dictionary_name}.jsonl"
    PATH_DATA = f"{PATH}/skills_matcher/data/{data_name}.csv"
    return PATH_DATA, PATH_DICT, PATH_TRAIN_DATA, PATH_VAL_DATA

# Load spacy_model.py

In [ ]:
PATH_DATA, PATH_DICT, PATH_TRAIN_DATA, PATH_VAL_DATA = load_paths('cleaned_data', 'doccano_dictionary', 'train', 'val')

In [ ]:
X_train, X_val = split_data(PATH_DICT, val_split=0.3)
spacy_train = make_spacy_model(X_train)
spacy_train.to_disk(PATH_TRAIN_DATA)
spacy_val = make_spacy_model(X_val)
spacy_val.to_disk(PATH_VAL_DATA)

Run the following command in the "models folder" to create a config file. If it is already created, no need to run again.

`python -m spacy init fill-config base_config.cfg config.cfg`

Run the following command in the "models" folder to train the model

`python -m spacy train config.cfg --output ./output_model`

## Training Test

In [ ]:
df = pd.read_csv(PATH_DATA)
text = df.description[845]

In [ ]:
entities = load_results_manual(df.description[3402])

In [ ]:
entities

# Try Spacy model with both annotations

In [ ]:
PATH_DATA_ALL, PATH_DICT_ALL, PATH_TRAIN_DATA_ALL, PATH_VAL_DATA_ALL = load_paths('cleaned_data', 'auto_label', 'train_auto', 'val_auto')

In [ ]:
X_train_all, X_val_all = split_data(PATH_DICT_ALL, val_split=0.3)

In [ ]:
spacy_train_all = make_spacy_model(X_train_all)
spacy_train_all.to_disk(PATH_TRAIN_DATA_ALL)
spacy_val_all = make_spacy_model(X_val_all)
spacy_val_all.to_disk(PATH_VAL_DATA_ALL)

In [ ]:
entities_auto = load_results_auto(df.description[3402])

## Add entities to Data Frame

In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
%%time
df['entities_auto_label'] = df.description.apply(load_results_auto)

In [ ]:
df_only_auto = df.copy()

In [ ]:
df_only_auto.to_csv('data_only_auto.csv')

In [3]:
df2 = pd.read_csv('data_only_auto.csv')

In [ ]:
%%time
df2['entities_manual_label'] = df2.description.apply(load_results_manual)